In [96]:
from rdfpandas.graph import to_graph
import pandas as pd
import rdflib
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from shexer.shaper import Shaper
from shexer.consts import NT, TURTLE

In [97]:
fdp_index = "https://index.vp.ejprarediseases.org/?format=ttl"

fdpGraph = rdflib.Graph()
fdpGraph.parse(fdp_index, format="ttl")

<Graph identifier=Nebc4f91c42104a2eae5d2b7a2a230a2c (<class 'rdflib.graph.Graph'>)>

In [98]:
print(fdpGraph.serialize(format="ttl"))

@prefix dcat: <http://www.w3.org/ns/dcat#> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix ldp: <http://www.w3.org/ns/ldp#> .
@prefix ns1: <https://w3id.org/fdp/fdp-o#> .
@prefix ns2: <http://www.re3data.org/schema/3-0#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://index.vp.ejprarediseases.org/catalog/> a ldp:DirectContainer ;
    dcterms:title "Catalogs" ;
    ldp:hasMemberRelation ns1:metadataCatalog ;
    ldp:membershipResource <https://index.vp.ejprarediseases.org> .

<https://index.vp.ejprarediseases.org#accessRights> a dcterms:RightsStatement ;
    dcterms:description "This resource has no access restriction" .

<https://index.vp.ejprarediseases.org#publisher> a foaf:Agent ;
    foaf:name "Default Publisher" .

<https://index.vp.ejprarediseases.org/profile/77aaad6a-0136-4c6e-88b9-07ffccd0ee4c> rdfs:label "FAIR Data Point Profile" .

<https://index.vp.ejp

# Issue 1
[ttl rendering of https://index.vp.ejprarediseases.org/](https://index.vp.ejprarediseases.org/?format=ttl) is not identical to html output on https://index.vp.ejprarediseases.org/

Those URL's need to be scraped which I will do below


In [104]:
url = 'https://index.vp.ejprarediseases.org'

headers = {'Accept': 'text/html'}
response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.content.decode('UTF-8'), 'html.parser')
text = soup.get_text()
for link in soup.find_all('a'):
    print(link.get('href'))

text

"FAIR Data PointWe're sorry but fdp-client doesn't work properly without JavaScript enabled. Please enable it to continue."

## Issue 2
The human-readable rendering of the FDP index can't be access programmatically, because it needs Javascript

In [105]:
# Launch a headless browser
driver = webdriver.Chrome()
driver.get(url)

# Wait for the page to fully load
driver.implicitly_wait(10)  # wait for up to 10 seconds

# Get the HTML content of the page
html = driver.page_source

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Close the browser


# Use BeautifulSoup to extract information from the page
htmls = []
for link in soup.find_all('a'):
    if "entry" in link.get('href'):
        print(link.get('href'))
        driver = webdriver.Chrome()
        driver.get(url+link.get('href'))
        driver.implicitly_wait(10)
        subhtml = driver.page_source
        sub_soup = BeautifulSoup(subhtml, 'html.parser')
        for sublink in sub_soup.find_all('a'):
            if "https" in sublink.get('href'):
                print(sublink.get('href'))
                fdpGraph.parse(sublink.get('href'), format="ttl")
                break

driver.quit()




/entry/d06f3341-bd37-426d-ad69-59fbf66ad043
https://w3id.org/fairvasc-fdp/
/entry/9e47792b-1540-4f4f-9748-db030f0787b1
/entry/255d928d-9f0b-4987-af9e-be9bf7d13392
https://w3id.org/smartcare-fdp
/entry/27d40e18-e1c6-45b1-940d-59105901bae8
https://w3id.org/ctsr-fdp


In [106]:
uris = {"fetched": [], "not_fetched": []}

print("Fetching linked data from the web")
for node in fdpGraph.all_nodes():
    if isinstance(node, rdflib.URIRef):
        try:

            print("Fetching: " + str(node))
            fdpGraph.parse(node)
            uris["fetched"].append(node)
        except:
            uris["not_fetched"].append(node)
            print("Could not parse: " + str(node))
            continue

Fetching linked data from the web
Fetching: http://www.ietf.org/rfc/rfc3986.txt
Could not parse: http://www.ietf.org/rfc/rfc3986.txt
Fetching: http://purl.org/dc/terms/RFC3066
Fetching: https://w3id.org/fairvasc-fdp/profile/77aaad6a-0136-4c6e-88b9-07ffccd0ee4c
Fetching: http://www.w3.org/ns/dcat#themeTaxonomy
Fetching: http://www.w3.org/ns/dcat#DataService
Fetching: http://rdflicense.appspot.com/rdflicense/cc-by-nc-nd3.0
Could not parse: http://rdflicense.appspot.com/rdflicense/cc-by-nc-nd3.0
Fetching: http://www.w3.org/ns/dcat#endDate
Fetching: http://purl.org/dc/terms/type
Fetching: http://purl.org/dc/terms/PhysicalResource
Fetching: http://purl.org/dc/terms/RFC5646
Fetching: http://www.w3.org/2002/07/owl#Ontology
Fetching: http://fadmaa.me/foaf.ttl
Could not parse: http://fadmaa.me/foaf.ttl
Fetching: http://purl.org/dc/elements/1.1/title
Could not parse: http://purl.org/dc/elements/1.1/title
Fetching: https://w3id.org/smartcare-fdp/metrics/445c0a70d1e214e545b261559e2842f4
Could not 

In [114]:
fdpGraph.serialize(format="ttl", destination="fdp_index.ttl")

<Graph identifier=Nebc4f91c42104a2eae5d2b7a2a230a2c (<class 'rdflib.graph.Graph'>)>

In [113]:
print("fetched:"+ str(len(uris["fetched"]))+", not fetched:"+ str(len(uris["not_fetched"])))

fetched:263, not fetched:142


In [115]:
shex_target_file = "fdp_index.shex"
target_classes = [
    "http://www.w3.org/ns/dcat#:Resource",
    "http://www.w3.org/ns/dcat#DataService",
    "https://w3id.org/fdp/fdp-o#MetadataService",
    "https://w3id.org/fdp/fdp-o#FAIRDataPoint"
]

shaper = Shaper(target_classes=target_classes,
                rdflib_graph=fdpGraph,
                input_format=TURTLE,
                )  # Default rdf:type

shaper.shex_graph(output_file="fdpSchema.shex")

In [116]:
!cat fdpSchema.shex

PREFIX : <http://weso.es/shapes/>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX csvw: <http://www.w3.org/ns/csvw#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dcam: <http://purl.org/dc/dcam/>
PREFIX doap: <http://usefulinc.com/ns/doap#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX odrl: <http://www.w3.org/ns/odrl/2/>
PREFIX org: <http://www.w3.org/ns/org#>
PREFIX qb: <http://purl.org/linked-data/cube#>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX ssn: <http://www.w3.org/ns/ssn/>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX vann: <http://purl.org/vocab/vann/>
PREFIX void: <http://rdfs.org/ns/void#>
PREFIX wgs: <https://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs

In [109]:
file = open("fdpSchema.shex", "r")
fdp_shex = file.read()
fdp_shex

'PREFIX : <http://weso.es/shapes/>\nPREFIX brick: <https://brickschema.org/schema/Brick#>\nPREFIX csvw: <http://www.w3.org/ns/csvw#>\nPREFIX dc: <http://purl.org/dc/elements/1.1/>\nPREFIX dcat: <http://www.w3.org/ns/dcat#>\nPREFIX dcam: <http://purl.org/dc/dcam/>\nPREFIX doap: <http://usefulinc.com/ns/doap#>\nPREFIX foaf: <http://xmlns.com/foaf/0.1/>\nPREFIX geo: <http://www.opengis.net/ont/geosparql#>\nPREFIX odrl: <http://www.w3.org/ns/odrl/2/>\nPREFIX org: <http://www.w3.org/ns/org#>\nPREFIX qb: <http://purl.org/linked-data/cube#>\nPREFIX sh: <http://www.w3.org/ns/shacl#>\nPREFIX skos: <http://www.w3.org/2004/02/skos/core#>\nPREFIX sosa: <http://www.w3.org/ns/sosa/>\nPREFIX ssn: <http://www.w3.org/ns/ssn/>\nPREFIX time: <http://www.w3.org/2006/time#>\nPREFIX vann: <http://purl.org/vocab/vann/>\nPREFIX void: <http://rdfs.org/ns/void#>\nPREFIX wgs: <https://www.w3.org/2003/01/geo/wgs84_pos#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-s

In [65]:
shapes = dict()
shapeName = None
with open('fdpSchema.shex', 'r') as f:
    for line in f:
        print(line)
        if line.startswith(":"):
            shapeName = line.replace(":", "").replace("\n", "")
            continue
        if shapeName is not None:
            shapes[shapeName] = shapes.get(shapeName, "") + line
        # code to do something with the list of numbers goes here
shapes

PREFIX : <http://weso.es/shapes/>

PREFIX brick: <https://brickschema.org/schema/Brick#>

PREFIX csvw: <http://www.w3.org/ns/csvw#>

PREFIX dc: <http://purl.org/dc/elements/1.1/>

PREFIX dcat: <http://www.w3.org/ns/dcat#>

PREFIX dcmitype: <http://purl.org/dc/dcmitype/>

PREFIX dcterms: <http://purl.org/dc/terms/>

PREFIX dcam: <http://purl.org/dc/dcam/>

PREFIX doap: <http://usefulinc.com/ns/doap#>

PREFIX foaf: <http://xmlns.com/foaf/0.1/>

PREFIX geo: <http://www.opengis.net/ont/geosparql#>

PREFIX odrl: <http://www.w3.org/ns/odrl/2/>

PREFIX org: <http://www.w3.org/ns/org#>

PREFIX prof: <http://www.w3.org/ns/dx/prof/>

PREFIX prov: <http://www.w3.org/ns/prov#>

PREFIX qb: <http://purl.org/linked-data/cube#>

PREFIX schema: <https://schema.org/>

PREFIX sh: <http://www.w3.org/ns/shacl#>

PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

PREFIX sosa: <http://www.w3.org/ns/sosa/>

PREFIX ssn: <http://www.w3.org/ns/ssn/>

PREFIX time: <http://www.w3.org/2006/time#>

PREFIX vann: <h

{'DataService': '{\n   dcterms:accessRights  IRI  ;                                # 100.0 %\n   rdf:type  [ns1:MetadataService]  ;                          # 100.0 %\n   rdf:type  [dcat:DataService]  ;                             # 100.0 %\n   rdf:type  [dcat:Resource]  ;                                # 100.0 %\n   rdf:type  [ns1:FAIRDataPoint]  ;                            # 100.0 %\n   ns2:repositoryIdentifier  IRI  ;                            # 100.0 %\n   ns1:metadataIssued  xsd:dateTime  ;                         # 100.0 %\n   dcterms:language  IRI  ;                                    # 100.0 %\n   dcterms:license  IRI  ;                                     # 100.0 %\n   ns1:fdpSoftwareVersion  xsd:string  ;                       # 100.0 %\n   ns1:metadataIdentifier  IRI  ;                              # 100.0 %\n   rdfs:label  xsd:string  ;                                   # 100.0 %\n   dcterms:publisher  IRI  ;                                   # 100.0 %\n   dcterms:conform

In [66]:
print(shapes['DataService'])

{
   dcterms:accessRights  IRI  ;                                # 100.0 %
   rdf:type  [ns1:MetadataService]  ;                          # 100.0 %
   rdf:type  [dcat:DataService]  ;                             # 100.0 %
   rdf:type  [dcat:Resource]  ;                                # 100.0 %
   rdf:type  [ns1:FAIRDataPoint]  ;                            # 100.0 %
   ns2:repositoryIdentifier  IRI  ;                            # 100.0 %
   ns1:metadataIssued  xsd:dateTime  ;                         # 100.0 %
   dcterms:language  IRI  ;                                    # 100.0 %
   dcterms:license  IRI  ;                                     # 100.0 %
   ns1:fdpSoftwareVersion  xsd:string  ;                       # 100.0 %
   ns1:metadataIdentifier  IRI  ;                              # 100.0 %
   rdfs:label  xsd:string  ;                                   # 100.0 %
   dcterms:publisher  IRI  ;                                   # 100.0 %
   dcterms:conformsTo  IRI  ;                    

In [110]:
import pandas as pd

# Split the input string into lines
lines = shapes['DataService'].split('\n')

# Extract the property name, value type, value constraints, and percentage from each line
data = []
for line in lines:
    line = line.strip()
    if not line or line.startswith('{') or line.startswith('}'):
        continue
    parts = line.split()
    prop_name = parts[0]
    value_type = parts[1]
    value_constraints = ' '.join(parts[2:-1]) if len(parts) > 2 else ''
    percentage = parts[-1]
    data.append((prop_name, value_type, value_constraints))

# Convert the list of tuples to a pandas dataframe
df = pd.DataFrame(data, columns=['Property Name', 'Value Type', 'Value Constraints'])

# Print the dataframe
df

,Property Name,Value Type,Value Constraints
0,dcterms:accessRights,IRI,; # 100.0
1,rdf:type,[ns1:MetadataService],; # 100.0
2,rdf:type,[dcat:DataService],; # 100.0
3,rdf:type,[dcat:Resource],; # 100.0
4,rdf:type,[ns1:FAIRDataPoint],; # 100.0
5,ns2:repositoryIdentifier,IRI,; # 100.0
6,ns1:metadataIssued,xsd:dateTime,; # 100.0
7,dcterms:language,IRI,; # 100.0
8,dcterms:license,IRI,; # 100.0
9,ns1:fdpSoftwareVersion,xsd:string,; # 100.0


In [86]:
for subject in fdpGraph.subjects():
    print(subject)

https://w3id.org/ctsr-fdp
https://w3id.org/ctsr-fdp
https://index.vp.ejprarediseases.org/catalog/
https://w3id.org/ctsr-fdp
https://w3id.org/fairvasc-fdp
https://w3id.org/fairvasc-fdp
https://w3id.org/ctsr-fdp
https://index.vp.ejprarediseases.org
https://index.vp.ejprarediseases.org
https://w3id.org/smartcare-fdp/catalog/
https://w3id.org/smartcare-fdp#identifier
https://w3id.org/smartcare-fdp#accessRights
https://w3id.org/fairvasc-fdp#identifier
https://w3id.org/ctsr-fdp
https://w3id.org/ctsr-fdp
https://index.vp.ejprarediseases.org
https://w3id.org/fairvasc-fdp
https://w3id.org/fairvasc-fdp/catalog/
https://w3id.org/fairvasc-fdp#publisher
https://w3id.org/smartcare-fdp
https://w3id.org/smartcare-fdp
https://w3id.org/ctsr-fdp/metrics/5d27e854a9e78eb3f663331cd47cdc13
https://index.vp.ejprarediseases.org/profile/77aaad6a-0136-4c6e-88b9-07ffccd0ee4c
https://index.vp.ejprarediseases.org
https://w3id.org/fairvasc-fdp
https://index.vp.ejprarediseases.org
https://index.vp.ejprarediseases.org